In [1]:
! pip install --no-cache-dir --upgrade git+https://github.com/evfro/polara.git@develop#egg=polara

  Cloning https://github.com/evfro/polara.git (to revision develop) to /tmp/pip-install-aig_we64/polara_30e62832cc74418fb20361fc4d13a9cc
  Running command git clone -q https://github.com/evfro/polara.git /tmp/pip-install-aig_we64/polara_30e62832cc74418fb20361fc4d13a9cc
  Running command git checkout -b develop --track origin/develop
  Switched to a new branch 'develop'
  Branch 'develop' set up to track remote branch 'develop' from 'origin'.
  Created wheel for polara: filename=polara-0.7.2.dev0-py3-none-any.whl size=87177 sha256=7e42a730b78f7301327b5f753f78a9c15c0d129e1821efa8c992edb21710ca4d
  Stored in directory: /tmp/pip-ephem-wheel-cache-gx_vb8wx/wheels/19/88/81/920c4189a6b0b92f19b02f18fcb44ded22ae852f7b09ab2e28
Successfully built polara


In [2]:
from polara import SVDModel
from polara.recommender.data import RecommenderData
import numpy as np
import pandas as pd
from tqdm import tqdm

## Data preprocessing and analysis 

In [27]:
memes = pd.read_csv('memes.csv', names=['meme_id', 'file_id', 'author_id', 'timestamp', 
                                        'filetype', 'unique_id', 'caption' ], header=None)
users_memes = pd.read_csv('users_memes.csv', names=['id__', 'chat_id', 'meme_id', 'reaction',
                                        'timestamp', 'message_id', 'timestamp_reaction'], header=None)
users = pd.read_csv('users.csv', names=['chat_id', 'name', '?', 'username',
                                        'date_add', 'is_fresh', '??'], header=None)

memes_with_clip = pd.read_csv('memes_wclip_feats.csv')

In [28]:
users.head()

,chat_id,name,?,username,date_add,is_fresh,??
0,259156066,Аринка-карантинка,259156066,chameleon_lizard,2021-12-10 22:22:39.000000,True,NaN
1,577915728,Спиридон,577915728,Blackaddder,2021-12-11 05:57:23.000000,True,NaN
2,439924549,oleg,439924549,ol3gka,2021-12-11 06:21:47.000000,True,NaN
3,411123163,MX,411123163,mx_bz,2022-01-19 07:31:46.000000,False,NaN
4,401426372,Аня,401426372,anvanche,2021-12-11 07:50:22.000000,True,NaN


In [29]:
users_memes.head()

,id__,chat_id,meme_id,reaction,timestamp,message_id,timestamp_reaction
0,1008,354637850,570,1002,2022-01-18 21:53:30.000000,7669,NaN
1,1018,354637850,597,1002,2022-01-18 23:36:17.000000,7689,NaN
2,1028,354637850,130,1002,2022-01-18 23:56:20.000000,7715,NaN
3,1038,354637850,2619,1002,2022-01-19 00:46:39.000000,7745,NaN
4,1048,354637850,576,1002,2022-01-19 01:10:00.000000,7769,NaN


In [30]:
memes_with_clip.head()

,meme_id,file_id,author_id,timestamp,filetype,unique_id,caption,funny_score,person_score,cat_score,...,meme_score,offensive_score,witty_score,cringe_score,people_score,anime_score,animated_score,emotional_score,cute_score,auto_score
0,2811,AgACAgQAAxkBAAJJWGHqZIz4f3WSyvH0XIFxhvPjwqWxAA...,2106431824,2022-01-21 07:45:16.000000,photo,NaN,NaN,0.250981,0.237676,0.217957,...,0.251079,0.213471,0.224981,0.229358,0.248814,0.223537,0.231514,0.252318,0.233501,0.249587
1,2826,AgACAgIAAxkBAAKIG2H3n_djkRg9MMUShOzCMHRcIm6mAA...,354637850,2022-01-31 08:38:15.000000,photo,AQADfL0xG4l3uUt9,NaN,0.264385,0.252193,0.220355,...,0.264848,0.243699,0.242989,0.242963,0.261154,0.244021,0.254006,0.247862,0.243598,0.252435
2,2827,AgACAgQAAxkBAAL2pGIEAAH2wjbF8DWHGiNewBn_UnrE6A...,354637850,2022-02-09 17:59:18.000000,photo,AQADyqoxGw8rrVF-,NaN,0.245900,0.212461,0.197702,...,0.239926,0.232262,0.242150,0.253753,0.212315,0.221327,0.222178,0.244417,0.230103,0.235808
3,2876,AgACAgQAAxkBAAL-aWIEBLnt94wyXCXhygRdYajavWbvAA...,2106431824,2022-02-09 18:15:21.000000,photo,AQADBqsxG3SVDFB-,"kvin, where is your corona",0.238833,0.237176,0.200625,...,0.240958,0.237480,0.239432,0.234064,0.240731,0.216932,0.225121,0.210288,0.224446,0.238979
4,2925,AgACAgQAAxkBAAEBAiBiBAetqQPwhJcXSKpCGXgI4lgbIw...,2106431824,2022-02-09 18:27:57.000000,photo,AQADUa0xGzmh5VJ9,Why do you keep coming back?,0.242307,0.206670,0.173449,...,0.245140,0.239347,0.234380,0.240613,0.207647,0.213957,0.219453,0.223919,0.223322,0.228427


In [65]:
clip_features = ['funny_score', 'person_score', 'cat_score', 'dog_score',
       'text_score', 'meme_score', 'offensive_score', 'witty_score',
       'cringe_score', 'people_score', 'anime_score', 'animated_score',
       'emotional_score', 'cute_score', 'auto_score']

In [66]:
memes_ids = memes_with_clip['meme_id'].unique()

In [67]:
np.sum(~memes.caption.isna())

2907

In [68]:
memes.head()

,meme_id,file_id,author_id,timestamp,filetype,unique_id,caption
0,2811,AgACAgQAAxkBAAJJWGHqZIz4f3WSyvH0XIFxhvPjwqWxAA...,2106431824,2022-01-21 07:45:16.000000,photo,NaN,NaN
1,2826,AgACAgIAAxkBAAKIG2H3n_djkRg9MMUShOzCMHRcIm6mAA...,354637850,2022-01-31 08:38:15.000000,photo,AQADfL0xG4l3uUt9,NaN
2,2827,AgACAgQAAxkBAAL2pGIEAAH2wjbF8DWHGiNewBn_UnrE6A...,354637850,2022-02-09 17:59:18.000000,photo,AQADyqoxGw8rrVF-,NaN
3,2876,AgACAgQAAxkBAAL-aWIEBLnt94wyXCXhygRdYajavWbvAA...,2106431824,2022-02-09 18:15:21.000000,photo,AQADBqsxG3SVDFB-,"kvin, where is your corona"
4,2925,AgACAgQAAxkBAAEBAiBiBAetqQPwhJcXSKpCGXgI4lgbIw...,2106431824,2022-02-09 18:27:57.000000,photo,AQADUa0xGzmh5VJ9,Why do you keep coming back?


In [69]:
data = users_memes[['chat_id', 'meme_id', 'reaction', 'timestamp']].copy()
data = data[data['meme_id'].isin(memes_ids)]
data.sort_values(by='timestamp', inplace=True) # sort data by time

REACTION2VALUE = {1001: -1, 107: -1, 106: -1, 105: -1, 104: -1,
                  103: 1, 102: 1, 101: 1, 100: 1, 1000: 1, 1002 : None} # 1002 means empty

data.reaction = users_memes.reaction.map(REACTION2VALUE)
data = data.dropna(axis='index', how='any', subset=['reaction'])

data.head()

,chat_id,meme_id,reaction,timestamp
882,354637850,398,-1.0,2021-12-08 20:35:19.000000
883,354637850,222,-1.0,2021-12-08 20:35:35.000000
884,481807223,387,1.0,2021-12-08 20:37:15.000000
885,354886383,61,-1.0,2021-12-08 20:39:05.000000
886,354886383,182,-1.0,2021-12-08 20:39:11.000000


In [70]:
train_data = data.copy()
train_data.drop_duplicates(subset=['chat_id', 'meme_id'], inplace=True)

In [71]:
train_data.reaction.value_counts()

-1.0    35854
 1.0    31493
Name: reaction, dtype: int64

In [72]:
print("num of unique users: ", len(train_data.chat_id.unique()))
print("num of unique memes: ", len(train_data.meme_id.unique()))
print("num of users that have at least 10 reactions: ", np.sum(train_data.chat_id.value_counts().values >= 10))

# let's leave only those users that have at least 10 reactions on memes
n_reactions = 10
train_data = train_data.loc[
    train_data
    .groupby('chat_id')['chat_id']
    .transform('count') >= n_reactions
]

num of unique users:  1057
num of unique memes:  5815
num of users that have at least 10 reactions:  865


In [73]:
data_model_1 = RecommenderData(train_data, 'chat_id', 'meme_id', 'reaction')
data_model_1.warm_start = False
data_model_1.holdout_size = 1
data_model_1.test_ratio = 0.2

train_val = data_model_1.training
holdout_test = data_model_1.test[1] #holdout
data_indexing_tvt = data_model_1.index


data_model_2 = RecommenderData(train_val, 'chat_id', 'meme_id', 'reaction')
data_model_2.warm_start = False
data_model_2.holdout_size = 1
data_model_2.test_ratio = 0.2

train = data_model_2.training
holdout_val = data_model_2.test[1] #holdout
data_indexing_tv = data_model_2.index



Preparing data...
5 unique meme_id entities within 5 holdout interactions were filtered. Reason: not in the training data.
Done.
There are 66353 events in the training and 168 events in the holdout.
Preparing data...
Users are not uniformly ordered! Unable to split test set reliably.
4 unique meme_id entities within 4 holdout interactions were filtered. Reason: not in the training data.
Done.
There are 66180 events in the training and 169 events in the holdout.


In [74]:
print(data_model_1._random_holdout, data_model_1._negative_prediction)

False False


In [75]:
print(f'train size = {train.shape[0]}, val size = {holdout_val.shape[0]}, test size = {holdout_test.shape[0]}')

train size = 66180, val size = 169, test size = 168


## Gradient Boosting with Clip fetures

In [98]:
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, roc_curve
from lightgbm import LGBMClassifier,plot_importance
from matplotlib import pyplot as plt
from itertools import product

from evaluation import topn_recommendations, model_evaluate, downvote_seen_items

In [99]:
data_description = dict(
    users = train['chat_id'].name,
    items = train['meme_id'].name,
    feedback = 'reaction',
    n_users = len(train['chat_id']),
    n_items = len(train['meme_id']),
    test_users = holdout_val[train['chat_id'].name].values
)

testset = data_description['test_users']
seen_data = train[train.chat_id.isin(testset)]

In [100]:
svd = SVDModel(data_model_2)
svd.rank = 10
svd.build()

item_factors = svd.factors['meme_id']
user_factors = svd.factors['chat_id']


PureSVD training time: 0.041s


In [101]:
item_reindexing_1 = data_model_1.index[1]
item_reindexing_1.sort_values(by='new', inplace=True)

item_reindexing_2 = data_model_2.index[1]
item_reindexing_2.sort_values(by='old', inplace=True)

full_reindexing = pd.DataFrame()
full_reindexing['old'] = item_reindexing_1['old']
full_reindexing['new'] = item_reindexing_2['new']


In [102]:
def change_item_factors(item_factors, meme_df, item_reindexing):
  add_factors = []
  for i in tqdm(range(len(item_factors))):
    old_id = item_reindexing[item_reindexing['new'] == i]['old'].values[0]
    addition = meme_df[meme_df['meme_id'] == old_id][clip_features].values[0]
    add_factors.append(addition)

  add_factors = np.array(add_factors)
  return np.concatenate((item_factors,add_factors), axis=1)



In [103]:
new_item_factors = change_item_factors(item_factors, memes_with_clip , full_reindexing)

100%|██████████| 5806/5806 [00:08<00:00, 681.09it/s]


In [104]:
def get_vectorized_data(df, user_factors, item_factors):
  X = []
  y_true = []
  for i in tqdm(range(df.shape[0])):
    user_features = user_factors[int(df.iloc[i]['chat_id']), :]
    item_features = item_factors[int(df.iloc[i]['meme_id']), :]
    features = np.concatenate((user_features,item_features), axis=0)
    y = (int(df.iloc[i]['reaction']) + 1) / 2 # here 1 stand for like and 0 for dislike
    y_true.append(y)
    X.append(features)
  X = np.array(X)

  return X, y_true


def boosting_scoring(model, holdout, user_factors, item_factors):
  scores = []
  for id_ in tqdm(holdout['chat_id'].values):
    user_features = user_factors[id_, :]
    user_matrix = item_factors.shape[0] * [user_features]
    user_matrix = np.array(user_matrix)
    features = np.concatenate((user_matrix, item_factors), axis=1)
    score = model.predict_proba(features)[:, 1]
    scores.append(score)
  return scores

In [105]:
X_train, y_train = get_vectorized_data(train, user_factors, new_item_factors)
X_val, y_val = get_vectorized_data(holdout_val, user_factors, new_item_factors)

100%|██████████| 169/169 [00:00<00:00, 1104.60it/s]


In [106]:
light = LGBMClassifier(n_estimators=200, learning_rate=0.01, max_depth=3)
light.fit(X_train, y_train)
scores = boosting_scoring(light, holdout_val, user_factors, new_item_factors)

100%|██████████| 169/169 [00:04<00:00, 37.29it/s]


In [107]:
downvote_seen_items(np.array(scores), seen_data, data_description)
recs = topn_recommendations(np.array(scores), topn=20)
print('HR={:.3}, MRR={:.3}, COV={:.3}'.format(*model_evaluate(recs, holdout_val, data_description,  topn=10)))

HR=0.0592, MRR=0.0159, COV=0.00178


## Search for best parameters for LightGBM+Clip pipline

In [108]:
ranks = [10, 50, 100, 300, 500]
nestims = [100, 200, 300, 400]
learning_rates = [0.01, 0.1]
max_depths = [3, 5, 7, 9]

params = product(ranks, nestims, learning_rates, max_depths)

In [109]:
results = []
for param in params:
  svd = SVDModel(data_model_2)
  svd.rank = param[0]
  svd.build()

  item_factors = svd.factors['meme_id']
  user_factors = svd.factors['chat_id']

  item_reindexing_1 = data_model_1.index[1]
  item_reindexing_1.sort_values(by='new', inplace=True)

  item_reindexing_2 = data_model_2.index[1]
  item_reindexing_2.sort_values(by='old', inplace=True)

  full_reindexing = pd.DataFrame()
  full_reindexing['old'] = item_reindexing_1['old']
  full_reindexing['new'] = item_reindexing_2['new']
  new_item_factors = change_item_factors(item_factors, memes_with_clip , full_reindexing)

  X_train, y_train = get_vectorized_data(train, user_factors, new_item_factors)

  light = LGBMClassifier(n_estimators=param[1], learning_rate=param[2], max_depth=param[3])
  light.fit(X_train, y_train)

  scores = boosting_scoring(light, holdout_val, user_factors, new_item_factors)

  downvote_seen_items(np.array(scores), seen_data, data_description)
  recs = topn_recommendations(np.array(scores), topn=10)
  print(param)
  hr, mrr, cov = model_evaluate(recs, holdout_val, data_description,  topn=10)
  print('HR={:.3}, MRR={:.3}, COV={:.3}'.format(*model_evaluate(recs, holdout_val, data_description,  topn=10)))

  results.append([*param, hr, mrr])



PureSVD training time: 0.044s


100%|██████████| 169/169 [00:02<00:00, 61.28it/s]


(10, 100, 0.01, 3)
HR=0.0237, MRR=0.00323, COV=0.0018
PureSVD training time: 0.057s


100%|██████████| 169/169 [00:03<00:00, 47.45it/s]


(10, 100, 0.01, 5)
HR=0.0828, MRR=0.0252, COV=0.00186
PureSVD training time: 0.041s


...

(100, 400, 0.1, 3)
HR=0.0118, MRR=0.0014, COV=0.00836
PureSVD training time: 0.272s


100%|██████████| 169/169 [00:19<00:00,  8.47it/s]


(100, 400, 0.1, 5)
HR=0.0118, MRR=0.00203, COV=0.00817
PureSVD training time: 0.271s


 25%|██▍       | 42/169 [00:05<00:15,  8.38it/s]


KeyboardInterrupt: ignored

In [123]:
best_params = {'rank' : 50,
               'n_estimators' : 200,
               'learning_rate' : 0.01,
               'max_depth' : 3}

data_description = dict(
    users = train_val['chat_id'].name,
    items = train_val['meme_id'].name,
    feedback = 'reaction',
    n_users = len(train_val['chat_id']),
    n_items = len(train_val['meme_id']),
    test_users = holdout_test[train_val['chat_id'].name].values
)

testset = data_description['test_users']
seen_data = train_val[train_val.chat_id.isin(testset)]
  
svd = SVDModel(data_model_1)
svd.rank = best_params['rank']
svd.build()

item_factors = svd.factors['meme_id']
user_factors = svd.factors['chat_id']

new_item_factors = change_item_factors(item_factors, memes_with_clip , item_reindexing_1)

X_train, y_train = get_vectorized_data(train_val, user_factors, new_item_factors)

light = LGBMClassifier(n_estimators=best_params['n_estimators'], learning_rate=best_params['learning_rate'], max_depth=best_params['max_depth'])
light.fit(X_train, y_train)

scores = boosting_scoring(light, holdout_test, user_factors, new_item_factors)

downvote_seen_items(np.array(scores), seen_data, data_description)
recs = topn_recommendations(np.array(scores), topn=10)
print(param)
hr, mrr, cov = model_evaluate(recs, holdout_test, data_description,  topn=10)
print('HR={:.3}, MRR={:.3}, COV={:.3}'.format(*model_evaluate(recs, holdout_test, data_description,  topn=10)))

PureSVD training time: 0.240s


100%|██████████| 168/168 [00:05<00:00, 30.60it/s]

(100, 400, 0.1, 7)
HR=0.0655, MRR=0.0155, COV=0.00188
